# PyTorch Optimization & Training: Making Neural Networks Learn

## 🎯 Introduction

Welcome to the heart of deep learning - the training process! This notebook will transform you from someone who can build neural networks into someone who can train them effectively. Understanding optimization is what separates toy examples from production-ready models.

### 🧠 What You'll Master

This comprehensive guide covers:
- **Training loop architecture**: The fundamental structure of neural network training
- **Optimizer comparison**: SGD, Adam, AdamW, and when to use each
- **Loss function selection**: Choosing the right objective for your problem
- **Learning rate strategies**: Schedules, warmup, and adaptive methods
- **Training stability**: Preventing divergence and ensuring convergence

### 🎓 Prerequisites

- Solid understanding of modules, parameters, and autograd
- Familiarity with neural network architecture concepts
- Basic knowledge of gradient descent and backpropagation

### 🚀 Why Training Mastery Matters

Effective training enables:
- **Convergence**: Reaching optimal solutions reliably
- **Efficiency**: Training faster with fewer resources
- **Stability**: Avoiding common training failures
- **Generalization**: Models that work on new data
- **Scalability**: Techniques that work from tiny to massive models

---

## 📚 Table of Contents

1. **[Training Loop Fundamentals](#training-loop-fundamentals)** - The core structure of neural network training
2. **[Optimizer Deep Dive](#optimizer-deep-dive)** - Understanding and comparing optimization algorithms
3. **[Loss Functions & Objectives](#loss-functions-objectives)** - Choosing the right training signal
4. **[Learning Rate Strategies](#learning-rate-strategies)** - Schedules and adaptive methods
5. **[Training Stability & Debugging](#training-stability-debugging)** - Ensuring reliable convergence

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Training Loop Fundamentals

### 🔄 The Heart of Deep Learning

Every neural network training follows the same fundamental pattern: forward pass, loss computation, backward pass, parameter update. Let's master this essential cycle and understand why each step matters!

In [ ]:
# =============================================================================
# THE COMPLETE TRAINING LOOP ARCHITECTURE
# =============================================================================

print("🔄 Training Loop Mastery")
print("=" * 50)

# Create a simple model and dataset for training demonstration
class SimpleRegressor(nn.Module):
    """Simple model for demonstrating training concepts."""
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(), 
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        return self.network(x)

# Generate synthetic dataset for training
torch.manual_seed(42)  # For reproducibility

# Create synthetic data: y = 2*x1 + 3*x2 + noise
n_samples, input_dim = 1000, 5
X = torch.randn(n_samples, input_dim)
# Only first two features matter, rest are noise
true_weights = torch.tensor([2.0, 3.0, 0.0, 0.0, 0.0])
y = X @ true_weights + 0.1 * torch.randn(n_samples)  # Add noise

print(f"Dataset created: {n_samples} samples, {input_dim} features")
print(f"True relationship: y = 2*x1 + 3*x2 + noise")

# Split into train/validation
train_size = int(0.8 * n_samples)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

print(f"Train set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")

# Initialize model, loss, and optimizer
model = SimpleRegressor(input_dim=input_dim, hidden_dim=32, output_dim=1)
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.01)

print(f"\nModel parameters: {sum(p.numel() for p in model.parameters()):,}")

print(f"\n🎯 The Universal Training Loop")
print("=" * 50)

# Training configuration
num_epochs = 100
batch_size = 32

# Create data loaders for batching
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Training loop with detailed commentary
train_losses = []
val_losses = []

print("Training progress (every 20 epochs):")
print("Epoch | Train Loss | Val Loss   | Notes")
print("------|------------|------------|-------------")

for epoch in range(num_epochs):
    # =================================
    # TRAINING PHASE
    # =================================
    model.train()  # Set to training mode (affects dropout, batchnorm)
    epoch_train_loss = 0.0
    num_train_batches = 0
    
    for batch_idx, (batch_X, batch_y) in enumerate(train_loader):
        # Step 1: Zero gradients from previous iteration
        # CRITICAL: Without this, gradients accumulate!
        optimizer.zero_grad()
        
        # Step 2: Forward pass - compute predictions
        predictions = model(batch_X)
        
        # Step 3: Compute loss between predictions and targets
        loss = criterion(predictions.squeeze(), batch_y)
        
        # Step 4: Backward pass - compute gradients
        loss.backward()
        
        # Step 5: Update parameters using computed gradients
        optimizer.step()
        
        # Track training progress
        epoch_train_loss += loss.item()
        num_train_batches += 1
    
    avg_train_loss = epoch_train_loss / num_train_batches
    train_losses.append(avg_train_loss)
    
    # =================================
    # VALIDATION PHASE
    # =================================
    model.eval()  # Set to evaluation mode
    epoch_val_loss = 0.0
    num_val_batches = 0
    
    with torch.no_grad():  # Disable gradient computation for efficiency
        for batch_X, batch_y in val_loader:
            predictions = model(batch_X)
            loss = criterion(predictions.squeeze(), batch_y)
            epoch_val_loss += loss.item()
            num_val_batches += 1
    
    avg_val_loss = epoch_val_loss / num_val_batches
    val_losses.append(avg_val_loss)
    
    # Progress reporting
    if epoch % 20 == 0 or epoch == num_epochs - 1:
        if epoch == 0:
            note = "Initial random weights"
        elif avg_val_loss < min(val_losses[:-1]):
            note = "New best validation!"
        elif avg_train_loss < 0.01:
            note = "Near convergence"
        else:
            note = "Training..."
            
        print(f"{epoch:5d} | {avg_train_loss:10.6f} | {avg_val_loss:10.6f} | {note}")

print(f"\n✅ Training completed!")
print(f"Final train loss: {train_losses[-1]:.6f}")
print(f"Final validation loss: {val_losses[-1]:.6f}")

# Check how well we learned the true relationship
learned_weights = model.network[0].weight[0].detach()  # First layer weights
print(f"\nLearned vs True weights (first 2 should be ~2.0, ~3.0):")
for i, (learned, true) in enumerate(zip(learned_weights, true_weights)):
    print(f"  Feature {i+1}: learned={learned.item():6.3f}, true={true.item():6.3f}")

print(f"\n🔍 Key Training Loop Elements")
print("=" * 50)
print("1. **model.train()**: Enable training mode (dropout, batchnorm active)")
print("2. **optimizer.zero_grad()**: Clear gradients from previous iteration")
print("3. **Forward pass**: Compute predictions from inputs")
print("4. **Loss computation**: Measure prediction quality")
print("5. **loss.backward()**: Compute gradients via backpropagation")
print("6. **optimizer.step()**: Update parameters using gradients")
print("7. **model.eval()**: Switch to evaluation mode for validation")
print("8. **torch.no_grad()**: Disable gradients during validation")

print(f"\n⚡ Performance Insights")
print("=" * 50)
best_val_epoch = val_losses.index(min(val_losses))
print(f"Best validation loss: {min(val_losses):.6f} at epoch {best_val_epoch}")
print(f"Training efficiency: {'High' if train_losses[-1] < 0.01 else 'Moderate'}")
print(f"Overfitting check: {'Good' if val_losses[-1] < 2 * min(val_losses) else 'Check'}")

In [ ]:
# =============================================================================
# MISSING DEFINITIONS FOR COMPREHENSIVE TRAINING DEMO
# =============================================================================

# SimpleClassifier class for classification tasks
class SimpleClassifier(nn.Module):
    """Simple neural network for multi-class classification."""
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(hidden_size // 2, num_classes)
        )
    
    def forward(self, x):
        return self.network(x)

# Universal training function
def train_model(model, train_loader, val_loader=None, num_epochs=10, lr=0.001, device='cpu'):
    """
    Universal training loop for classification tasks.
    
    Args:
        model: PyTorch model to train
        train_loader: DataLoader for training data
        val_loader: DataLoader for validation data (optional)
        num_epochs: Number of training epochs
        lr: Learning rate
        device: Device to train on ('cpu' or 'cuda')
    
    Returns:
        Dictionary with training history
    """
    model = model.to(device)
    
    # Setup loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    
    # Training history
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }
    
    print(f"Training for {num_epochs} epochs on {device}")
    print("Epoch | Train Loss | Train Acc | Val Loss | Val Acc")
    print("------|------------|-----------|----------|--------")
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(data)
            loss = criterion(outputs, target)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += target.size(0)
            correct_train += (predicted == target).sum().item()
        
        # Calculate training metrics
        epoch_train_loss = running_loss / len(train_loader)
        epoch_train_acc = 100.0 * correct_train / total_train
        
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        
        # Validation phase
        epoch_val_loss = 0.0
        epoch_val_acc = 0.0
        
        if val_loader is not None:
            model.eval()
            val_loss = 0.0
            correct_val = 0
            total_val = 0
            
            with torch.no_grad():
                for data, target in val_loader:
                    data, target = data.to(device), target.to(device)
                    outputs = model(data)
                    loss = criterion(outputs, target)
                    
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    total_val += target.size(0)
                    correct_val += (predicted == target).sum().item()
            
            epoch_val_loss = val_loss / len(val_loader)
            epoch_val_acc = 100.0 * correct_val / total_val
            
            history['val_loss'].append(epoch_val_loss)
            history['val_acc'].append(epoch_val_acc)
        
        # Print progress
        if val_loader is not None:
            print(f"{epoch+1:5d} | {epoch_train_loss:10.4f} | {epoch_train_acc:8.2f}% | {epoch_val_loss:8.4f} | {epoch_val_acc:7.2f}%")
        else:
            print(f"{epoch+1:5d} | {epoch_train_loss:10.4f} | {epoch_train_acc:8.2f}% |    N/A   |   N/A")
    
    return history

# Sample dataset creation for classification
def create_classification_dataset(n_samples=1000, n_features=20, n_classes=5, noise=0.1):
    """
    Create a synthetic classification dataset.
    
    Args:
        n_samples: Number of samples to generate
        n_features: Number of input features
        n_classes: Number of classes
        noise: Amount of noise to add
    
    Returns:
        X: Input features
        y: Class labels
    """
    torch.manual_seed(42)  # For reproducibility
    
    # Generate random centers for each class
    class_centers = torch.randn(n_classes, n_features) * 2
    
    # Generate samples
    X = torch.zeros(n_samples, n_features)
    y = torch.zeros(n_samples, dtype=torch.long)
    
    samples_per_class = n_samples // n_classes
    
    for class_idx in range(n_classes):
        start_idx = class_idx * samples_per_class
        end_idx = start_idx + samples_per_class
        
        if class_idx == n_classes - 1:  # Last class gets remaining samples
            end_idx = n_samples
        
        # Generate samples around class center
        class_samples = class_centers[class_idx].unsqueeze(0) + noise * torch.randn(end_idx - start_idx, n_features)
        
        X[start_idx:end_idx] = class_samples
        y[start_idx:end_idx] = class_idx
    
    # Shuffle the dataset
    perm = torch.randperm(n_samples)
    X = X[perm]
    y = y[perm]
    
    return X, y

# Create the classification dataset
print("Creating classification dataset...")
X_class, y_class = create_classification_dataset(n_samples=2000, n_features=20, n_classes=5)

# Split into train/validation
train_size = int(0.8 * len(X_class))
X_train_class = X_class[:train_size]
y_train_class = y_class[:train_size]
X_val_class = X_class[train_size:]
y_val_class = y_class[train_size:]

print(f"Classification dataset created:")
print(f"  Total samples: {len(X_class)}")
print(f"  Features: {X_class.shape[1]}")
print(f"  Classes: {len(torch.unique(y_class))}")
print(f"  Train samples: {len(X_train_class)}")
print(f"  Validation samples: {len(X_val_class)}")

# Create data loaders
train_dataset_class = TensorDataset(X_train_class, y_train_class)
train_loader = DataLoader(train_dataset_class, batch_size=32, shuffle=True)

val_dataset_class = TensorDataset(X_val_class, y_val_class)
val_loader = DataLoader(val_dataset_class, batch_size=32, shuffle=False)

print(f"Data loaders created with batch size 32")
print(f"  Training batches: {len(train_loader)}")
print(f"  Validation batches: {len(val_loader)}")

# Visualization function for training curves  
def plot_training_curves_detailed(history):
    """
    Plot comprehensive training curves with additional insights.
    
    Args:
        history: Dictionary with training metrics
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss curves
    axes[0, 0].plot(epochs, history['train_loss'], 'b-', label='Training Loss', linewidth=2)
    if history['val_loss']:
        axes[0, 0].plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Training and Validation Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Accuracy curves
    axes[0, 1].plot(epochs, history['train_acc'], 'b-', label='Training Accuracy', linewidth=2)
    if history['val_acc']:
        axes[0, 1].plot(epochs, history['val_acc'], 'r-', label='Validation Accuracy', linewidth=2)
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].set_title('Training and Validation Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Loss difference (overfitting indicator)
    if history['val_loss']:
        loss_diff = [val - train for train, val in zip(history['train_loss'], history['val_loss'])]
        axes[1, 0].plot(epochs, loss_diff, 'g-', linewidth=2)
        axes[1, 0].axhline(y=0, color='k', linestyle='--', alpha=0.5)
        axes[1, 0].set_xlabel('Epoch')
        axes[1, 0].set_ylabel('Validation Loss - Training Loss')
        axes[1, 0].set_title('Overfitting Monitor (Higher = More Overfitting)')
        axes[1, 0].grid(True, alpha=0.3)
    else:
        axes[1, 0].text(0.5, 0.5, 'No validation data\\nfor overfitting analysis', 
                       ha='center', va='center', transform=axes[1, 0].transAxes, fontsize=12)
        axes[1, 0].set_title('Overfitting Monitor')
    
    # Learning progress (accuracy improvement)
    if len(history['train_acc']) > 1:
        train_acc_diff = [history['train_acc'][i] - history['train_acc'][i-1] 
                         for i in range(1, len(history['train_acc']))]
        axes[1, 1].plot(epochs[1:], train_acc_diff, 'b-', alpha=0.7, label='Train Acc Improvement')
        
        if history['val_acc'] and len(history['val_acc']) > 1:
            val_acc_diff = [history['val_acc'][i] - history['val_acc'][i-1] 
                           for i in range(1, len(history['val_acc']))]
            axes[1, 1].plot(epochs[1:], val_acc_diff, 'r-', alpha=0.7, label='Val Acc Improvement')
        
        axes[1, 1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
        axes[1, 1].set_xlabel('Epoch')
        axes[1, 1].set_ylabel('Accuracy Improvement (%)')
        axes[1, 1].set_title('Learning Progress')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    else:
        axes[1, 1].text(0.5, 0.5, 'Need more epochs\\nfor progress analysis', 
                       ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
        axes[1, 1].set_title('Learning Progress')
    
    plt.tight_layout()
    plt.show()
    
    # Print comprehensive metrics
    print("\\n" + "="*60)
    print("TRAINING SUMMARY")
    print("="*60)
    
    print(f"Final Training Loss: {history['train_loss'][-1]:.4f}")
    print(f"Final Training Accuracy: {history['train_acc'][-1]:.2f}%")
    
    if history['val_loss']:
        print(f"Final Validation Loss: {history['val_loss'][-1]:.4f}")
        print(f"Final Validation Accuracy: {history['val_acc'][-1]:.2f}%")
        
        # Best validation performance
        best_val_epoch = np.argmax(history['val_acc'])
        best_val_acc = history['val_acc'][best_val_epoch]
        best_val_loss = history['val_loss'][best_val_epoch]
        
        print(f"Best Validation Accuracy: {best_val_acc:.2f}% (Epoch {best_val_epoch + 1})")
        print(f"Best Validation Loss: {best_val_loss:.4f} (Epoch {best_val_epoch + 1})")
        
        # Overfitting analysis
        final_gap = history['val_loss'][-1] - history['train_loss'][-1]
        print(f"Final Train-Val Loss Gap: {final_gap:.4f}")
        
        if final_gap > 0.5:
            print("⚠️  Potential overfitting detected!")
        elif final_gap < 0.1:
            print("✅ Good generalization")
        else:
            print("📊 Moderate generalization")
    
    # Training efficiency
    if len(history['train_acc']) >= 2:
        total_improvement = history['train_acc'][-1] - history['train_acc'][0]
        print(f"Total Training Improvement: {total_improvement:.2f}%")
        
        if total_improvement > 20:
            print("🚀 Excellent learning progress")
        elif total_improvement > 10:
            print("✅ Good learning progress")
        else:
            print("📈 Moderate learning progress")

print("\\n✅ All missing definitions added successfully!")
print("The notebook is now ready for comprehensive training demonstrations.")

In [ ]:
# Plot training curves with comprehensive analysis
plot_training_curves_detailed(history)

In [ ]:
# Plot training curves
def plot_training_curves(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    epochs = range(1, len(history['train_loss']) + 1)
    
    # Loss curves
    ax1.plot(epochs, history['train_loss'], 'b-', label='Training Loss')
    if history['val_loss']:
        ax1.plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Training and Validation Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Accuracy curves
    ax2.plot(epochs, history['train_acc'], 'b-', label='Training Accuracy')
    if history['val_acc']:
        ax2.plot(epochs, history['val_acc'], 'r-', label='Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy (%)')
    ax2.set_title('Training and Validation Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print final metrics
    print(f"Final Training Loss: {history['train_loss'][-1]:.4f}")
    print(f"Final Training Accuracy: {history['train_acc'][-1]:.2f}%")
    if history['val_loss']:
        print(f"Final Validation Loss: {history['val_loss'][-1]:.4f}")
        print(f"Final Validation Accuracy: {history['val_acc'][-1]:.2f}%")

plot_training_curves(history)

## Optimizers: SGD vs AdamW

In [ ]:
# Compare different optimizers
def compare_optimizers():
    """Compare SGD, Adam, and AdamW optimizers"""
    
    # Create identical models
    models = {
        'SGD': SimpleClassifier(20, 128, 5),
        'Adam': SimpleClassifier(20, 128, 5),
        'AdamW': SimpleClassifier(20, 128, 5)
    }
    
    # Make sure they start with identical weights
    state_dict = models['SGD'].state_dict()
    for model in models.values():
        model.load_state_dict(state_dict)
    
    # Create optimizers
    optimizers = {
        'SGD': optim.SGD(models['SGD'].parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4),
        'Adam': optim.Adam(models['Adam'].parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-4),
        'AdamW': optim.AdamW(models['AdamW'].parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-4)
    }
    
    criterion = nn.CrossEntropyLoss()
    
    # Track progress for each optimizer
    histories = {name: {'loss': [], 'acc': []} for name in models.keys()}
    
    num_epochs = 15
    
    for epoch in range(num_epochs):
        for name in models.keys():
            model = models[name]
            optimizer = optimizers[name]
            
            model.train()
            total_loss = 0
            correct = 0
            total = 0
            
            for data, target in train_loader:
                optimizer.zero_grad()
                output = model(data)
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
            
            avg_loss = total_loss / len(train_loader)
            accuracy = 100.0 * correct / total
            
            histories[name]['loss'].append(avg_loss)
            histories[name]['acc'].append(accuracy)
        
        if (epoch + 1) % 5 == 0:
            print(f"Epoch {epoch+1:2d}:")
            for name in models.keys():
                loss = histories[name]['loss'][-1]
                acc = histories[name]['acc'][-1]
                print(f"  {name:5s}: Loss = {loss:.4f}, Acc = {acc:.2f}%")
    
    return histories

# Compare optimizers
print("Comparing optimizers...")
optimizer_histories = compare_optimizers()

# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

epochs = range(1, len(optimizer_histories['SGD']['loss']) + 1)
colors = {'SGD': 'blue', 'Adam': 'green', 'AdamW': 'red'}

# Loss comparison
for name, history in optimizer_histories.items():
    ax1.plot(epochs, history['loss'], color=colors[name], label=f'{name}', marker='o', markersize=3)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Optimizer Comparison: Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Accuracy comparison
for name, history in optimizer_histories.items():
    ax2.plot(epochs, history['acc'], color=colors[name], label=f'{name}', marker='o', markersize=3)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Optimizer Comparison: Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print final comparison
print("\nFinal Results:")
for name, history in optimizer_histories.items():
    final_loss = history['loss'][-1]
    final_acc = history['acc'][-1]
    print(f"{name:5s}: Loss = {final_loss:.4f}, Accuracy = {final_acc:.2f}%")

print("\nKey differences:")
print("• SGD: Simple, requires learning rate tuning, benefits from momentum")
print("• Adam: Adaptive learning rates, fast convergence, can overfit")
print("• AdamW: Adam with decoupled weight decay, better generalization")

## Common Loss Functions

In [ ]:
# Demonstrate different loss functions
print("=== Common Loss Functions Demo ===")

# 1. Classification: CrossEntropyLoss
print("\n1. CrossEntropyLoss (Multi-class Classification)")
ce_loss = nn.CrossEntropyLoss()

# Create sample data
batch_size, num_classes = 4, 5
logits = torch.randn(batch_size, num_classes)  # Raw scores from model
targets = torch.randint(0, num_classes, (batch_size,))  # Class indices

loss_ce = ce_loss(logits, targets)
print(f"Logits shape: {logits.shape}")
print(f"Targets shape: {targets.shape}")
print(f"Targets: {targets}")
print(f"CrossEntropyLoss: {loss_ce.item():.4f}")

# Show what happens with perfect predictions
perfect_logits = torch.zeros_like(logits)
for i, target in enumerate(targets):
    perfect_logits[i, target] = 10.0  # High score for correct class
perfect_loss = ce_loss(perfect_logits, targets)
print(f"Perfect prediction loss: {perfect_loss.item():.4f}")

# 2. Regression: MSELoss
print("\n2. MSELoss (Regression)")
mse_loss = nn.MSELoss()

predictions = torch.randn(batch_size, 1)
true_values = torch.randn(batch_size, 1)

loss_mse = mse_loss(predictions, true_values)
print(f"Predictions: {predictions.squeeze()}")
print(f"True values: {true_values.squeeze()}")
print(f"MSELoss: {loss_mse.item():.4f}")

# Show difference between MSE and MAE
mae_loss = nn.L1Loss()
loss_mae = mae_loss(predictions, true_values)
print(f"MAE (L1) Loss: {loss_mae.item():.4f}")

# 3. Binary Classification: BCEWithLogitsLoss
print("\n3. BCEWithLogitsLoss (Binary Classification)")
bce_loss = nn.BCEWithLogitsLoss()

binary_logits = torch.randn(batch_size, 1)  # Raw logits
binary_targets = torch.randint(0, 2, (batch_size, 1)).float()  # 0 or 1

loss_bce = bce_loss(binary_logits, binary_targets)
print(f"Binary logits: {binary_logits.squeeze()}")
print(f"Binary targets: {binary_targets.squeeze()}")
print(f"BCEWithLogitsLoss: {loss_bce.item():.4f}")

# Compare with manual BCE calculation
probs = torch.sigmoid(binary_logits)
print(f"Converted to probabilities: {probs.squeeze()}")

# 4. Negative Log Likelihood: NLLLoss
print("\n4. NLLLoss (when you already have log probabilities)")
nll_loss = nn.NLLLoss()

# NLLLoss expects log probabilities
log_probs = F.log_softmax(logits, dim=1)
loss_nll = nll_loss(log_probs, targets)

print(f"Log probabilities shape: {log_probs.shape}")
print(f"NLLLoss: {loss_nll.item():.4f}")
print(f"Note: CrossEntropyLoss = LogSoftmax + NLLLoss")
print(f"Verification - CE loss: {loss_ce.item():.4f}, NLL loss: {loss_nll.item():.4f}")
print(f"Match: {abs(loss_ce.item() - loss_nll.item()) < 1e-6}")

# 5. Multi-label classification: BCEWithLogitsLoss with multiple outputs
print("\n5. Multi-label Classification")
num_labels = 3
multi_logits = torch.randn(batch_size, num_labels)
multi_targets = torch.randint(0, 2, (batch_size, num_labels)).float()  # Multiple binary labels

multi_bce_loss = nn.BCEWithLogitsLoss()
loss_multi = multi_bce_loss(multi_logits, multi_targets)

print(f"Multi-label logits shape: {multi_logits.shape}")
print(f"Multi-label targets shape: {multi_targets.shape}")
print(f"Multi-label BCE loss: {loss_multi.item():.4f}")
print(f"Sample targets: {multi_targets[0]}  (can have multiple 1s)")

## Train vs Eval Modes

In [ ]:
# Demonstrate train vs eval mode differences
print("=== Train vs Eval Mode Demo ===")

# Create model with dropout and batch norm
class ModelWithDropoutAndBN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.5)  # 50% dropout
        
        self.fc2 = nn.Linear(hidden_size, hidden_size//2)
        self.bn2 = nn.BatchNorm1d(hidden_size//2)
        self.dropout2 = nn.Dropout(0.3)  # 30% dropout
        
        self.fc3 = nn.Linear(hidden_size//2, output_size)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        
        x = self.fc3(x)
        return x

model = ModelWithDropoutAndBN(20, 128, 5)
x = torch.randn(10, 20)  # Batch of 10 samples

print("1. Dropout behavior:")
print("=" * 30)

# Training mode: dropout is active
model.train()
print("Training mode (dropout active):")
out1 = model(x)
out2 = model(x)  # Same input, different output due to dropout

print(f"Output 1 mean: {out1.mean().item():.4f}")
print(f"Output 2 mean: {out2.mean().item():.4f}")
print(f"Outputs are different: {not torch.allclose(out1, out2)}")

# Evaluation mode: dropout is disabled
model.eval()
print("\nEvaluation mode (dropout disabled):")
out3 = model(x)
out4 = model(x)  # Same input, same output

print(f"Output 3 mean: {out3.mean().item():.4f}")
print(f"Output 4 mean: {out4.mean().item():.4f}")
print(f"Outputs are identical: {torch.allclose(out3, out4)}")

print("\n2. BatchNorm behavior:")
print("=" * 30)

# BatchNorm tracks running statistics differently in train vs eval
def check_bn_stats(model, mode_name):
    bn_layer = model.bn1
    print(f"{mode_name} mode:")
    print(f"  Running mean: {bn_layer.running_mean[:5]}")
    print(f"  Running var:  {bn_layer.running_var[:5]}")
    print(f"  Training: {bn_layer.training}")

# Reset batch norm statistics
def reset_bn_stats(model):
    for module in model.modules():
        if isinstance(module, nn.BatchNorm1d):
            module.reset_running_stats()

reset_bn_stats(model)

# Training mode: updates running statistics
model.train()
_ = model(x)
check_bn_stats(model, "Training")

# Evaluation mode: uses fixed running statistics
model.eval()
old_running_mean = model.bn1.running_mean.clone()
_ = model(x)
check_bn_stats(model, "Evaluation")

print(f"Running mean changed in eval: {not torch.allclose(old_running_mean, model.bn1.running_mean)}")

print("\n3. Practical implications:")
print("=" * 30)
print("✓ Always call model.train() before training")
print("✓ Always call model.eval() before inference")
print("✓ Use torch.no_grad() during inference to save memory")
print("✓ Dropout provides regularization during training")
print("✓ BatchNorm uses batch statistics in training, running stats in eval")

In [ ]:
# Demonstrate correct inference pattern
def inference_example():
    """Show proper inference setup"""
    
    # Assume we have a trained model
    model = SimpleClassifier(20, 64, 5)
    model.eval()  # Set to evaluation mode
    
    # Sample test data
    test_data = torch.randn(100, 20)
    
    print("Inference setup:")
    
    # Method 1: Basic inference
    with torch.no_grad():  # Disable gradient computation
        predictions = model(test_data)
        probabilities = F.softmax(predictions, dim=1)
        predicted_classes = torch.argmax(predictions, dim=1)
    
    print(f"Predictions shape: {predictions.shape}")
    print(f"Sample probabilities: {probabilities[0]}")
    print(f"Predicted classes: {predicted_classes[:10]}")
    
    # Method 2: Batch processing for large datasets
    def batch_inference(model, data, batch_size=32):
        model.eval()
        all_predictions = []
        
        with torch.no_grad():
            for i in range(0, len(data), batch_size):
                batch = data[i:i+batch_size]
                batch_pred = model(batch)
                all_predictions.append(batch_pred)
        
        return torch.cat(all_predictions, dim=0)
    
    # Process large dataset in batches
    large_test_data = torch.randn(1000, 20)
    batch_predictions = batch_inference(model, large_test_data, batch_size=64)
    
    print(f"\nBatch inference on {len(large_test_data)} samples:")
    print(f"Output shape: {batch_predictions.shape}")
    
    # Memory usage comparison
    import torch.cuda
    if torch.cuda.is_available():
        print("\nMemory usage patterns:")
        print("• torch.no_grad() reduces memory usage by ~2x")
        print("• Batch processing prevents OOM on large datasets")
        print("• model.eval() ensures consistent results")

inference_example()

## Learning Rate Scheduling

In [ ]:
# Demonstrate learning rate scheduling
print("=== Learning Rate Scheduling Demo ===")

def demonstrate_lr_scheduling():
    """Show different learning rate scheduling strategies"""
    
    # Create a simple model and optimizer
    model = SimpleClassifier(20, 64, 5)
    base_lr = 0.1
    optimizer = optim.SGD(model.parameters(), lr=base_lr)
    
    num_epochs = 50
    
    # Different schedulers
    schedulers = {
        'StepLR': optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5),
        'ExponentialLR': optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95),
        'CosineAnnealing': optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.001),
        'ReduceLROnPlateau': optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)
    }
    
    # Track learning rates for each scheduler
    lr_histories = {name: [] for name in schedulers.keys()}
    lr_histories['No Scheduling'] = []
    
    # Simulate training with different schedulers
    for name, scheduler in schedulers.items():
        # Reset optimizer
        optimizer = optim.SGD(model.parameters(), lr=base_lr)
        if name != 'ReduceLROnPlateau':
            scheduler = type(scheduler)(optimizer, **scheduler.state_dict())
        else:
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)
        
        for epoch in range(num_epochs):
            current_lr = optimizer.param_groups[0]['lr']
            lr_histories[name].append(current_lr)
            
            # For ReduceLROnPlateau, we need to provide a metric
            if name == 'ReduceLROnPlateau':
                # Simulate a loss that decreases then plateaus
                fake_loss = 1.0 * np.exp(-epoch/10) + 0.1 + 0.05 * np.random.random()
                if epoch > 20:  # Start plateauing
                    fake_loss = 0.15 + 0.02 * np.random.random()
                scheduler.step(fake_loss)
            else:
                scheduler.step()
    
    # Add no scheduling baseline
    lr_histories['No Scheduling'] = [base_lr] * num_epochs
    
    return lr_histories

# Generate learning rate histories
lr_histories = demonstrate_lr_scheduling()

# Plot learning rate schedules
plt.figure(figsize=(12, 6))

epochs = range(len(lr_histories['No Scheduling']))
colors = ['blue', 'green', 'red', 'orange', 'purple']

for i, (name, lr_history) in enumerate(lr_histories.items()):
    plt.plot(epochs, lr_history, label=name, color=colors[i], linewidth=2)

plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Scheduling Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Log scale to see all schedules clearly
plt.show()

# Explain each scheduler
print("\nScheduler explanations:")
print("=" * 40)
print("StepLR: Reduces LR by factor γ every step_size epochs")
print("ExponentialLR: Multiplies LR by γ each epoch")
print("CosineAnnealing: Follows cosine curve from max to min LR")
print("ReduceLROnPlateau: Reduces LR when metric stops improving")
print("No Scheduling: Constant learning rate")

print("\nWhen to use each:")
print("• StepLR: Simple, works well with SGD")
print("• ExponentialLR: Smooth decay, good for long training")
print("• CosineAnnealing: Popular for transformers, smooth restart")
print("• ReduceLROnPlateau: Adaptive, responds to actual performance")

In [ ]:
# Advanced: Learning rate warmup
class WarmupScheduler:
    """Learning rate warmup followed by decay"""
    
    def __init__(self, optimizer, warmup_epochs, max_lr, total_epochs):
        self.optimizer = optimizer
        self.warmup_epochs = warmup_epochs
        self.max_lr = max_lr
        self.total_epochs = total_epochs
        self.current_epoch = 0
    
    def step(self):
        if self.current_epoch < self.warmup_epochs:
            # Linear warmup
            lr = self.max_lr * (self.current_epoch + 1) / self.warmup_epochs
        else:
            # Cosine decay after warmup
            progress = (self.current_epoch - self.warmup_epochs) / (self.total_epochs - self.warmup_epochs)
            lr = 0.5 * self.max_lr * (1 + np.cos(np.pi * progress))
        
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        
        self.current_epoch += 1
        return lr

# Demonstrate warmup scheduling
model = SimpleClassifier(20, 64, 5)
optimizer = optim.AdamW(model.parameters(), lr=0.001)  # Will be overridden

warmup_scheduler = WarmupScheduler(
    optimizer=optimizer,
    warmup_epochs=10,
    max_lr=0.01,
    total_epochs=100
)

# Track warmup schedule
warmup_lrs = []
for epoch in range(100):
    lr = warmup_scheduler.step()
    warmup_lrs.append(lr)

# Plot warmup schedule
plt.figure(figsize=(10, 4))
plt.plot(range(100), warmup_lrs, 'b-', linewidth=2, label='Warmup + Cosine Decay')
plt.axvline(x=10, color='r', linestyle='--', alpha=0.7, label='End of Warmup')
plt.xlabel('Epoch')
plt.ylabel('Learning Rate')
plt.title('Learning Rate Warmup + Cosine Decay')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Warmup benefits:")
print("• Prevents early training instability")
print("• Especially important for large batch sizes")
print("• Common in transformer training")
print("• Allows higher maximum learning rates")

print("\n🎉 Optimization & Training exploration completed!")
print("\nKey takeaways:")
print("• Use a systematic training loop structure")
print("• AdamW is generally a good default optimizer")
print("• Match loss function to your task type")
print("• Always set model.train()/model.eval() appropriately")
print("• Learning rate scheduling can significantly improve results")
print("• Monitor both training and validation metrics")